# Inverted Pendulum on a Drone

This notebook is used to design a controller for the inverted pendulum on a Crazyflie system. The notebook goes over the derivation of the dynamics of a simplified system and plugs in these equations into the full dynamics of the drone. Using a linearized state-space version of the full system (now with 14 states that represent the drone and the pendulum), the controller for the drone is designed to balance the pendulum in its upright position.

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg
import scipy
from scipy import linalg
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
import sympy as sym
from sympy.physics.mechanics import *

# 1. Measure Moments of Inertia

This section calculates the moments of inertia of the drone with the pendulum in the fully upright position.

In [3]:
def load_hardware_data(filename, t_min_offset=0, t_max_offset=0):
    # load raw data
    with open(filename, 'r') as f:
        data = json.load(f)

    # convert lists to numpy arrays
    for val in data.values():
        for key in val.keys():
            val[key] = np.array(val[key])

    # create an array of times at which to subsample
    t_min = -np.inf
    t_max = np.inf
    for key, val in data.items():
        t_min = max(t_min, val['time'][0])
        t_max = min(t_max, val['time'][-1])
    t_min += t_min_offset * 1000
    t_max -= t_max_offset * 1000
    nt = int(1 + np.floor((t_max - t_min) / 10.))
    t = np.arange(0, 10 * nt, 10) / 1000.
    resampled_data = {'time': t}

    # resample raw data with linear interpolation
    for k, v in data.items():
        f = interp1d((v['time'] - t_min) / 1000., v['data'])
        resampled_data[k] = f(t)
        
    # return the resampled data
    return resampled_data

### 1.1 Define constant values

In [4]:
g = 9.81 # gravity (m/s)
m = 0.044 # mass (kg)
r = 0.03 # x- and y-axis moment arm
r_z = 0.015 # z-axis moment arm

### 1.2 Perform x-axis moment of inertia calculation

In [5]:
data = load_hardware_data(
    'x_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)

t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])

# Find the index of each peak (increase "prominence" if you get bad results)
peaks = find_peaks(w_x, prominence=0)
i_peaks = peaks[0]

# Find the time at each peak
t_peaks = t[i_peaks]

# Find w_x at each peak (for visualization)
w_x_peaks = w_x[i_peaks]

t_diff = t_peaks[1:] - t_peaks[:-1]
t_diff = t_diff[10:]
# Find the mean difference as an estimate of the oscillation period
T = np.mean(t_diff)

J_x = (m * g * r/((2 * np.pi / T) ** 2)) - m * r ** 2 # <-- FIXME
print(J_x)

1.8260446760861986e-05


### 1.3 Perform y-axis moment of inertia calculation

In [6]:
data = load_hardware_data(
    'y_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)
t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])
peaks = find_peaks(w_y, prominence=0)
i_peaks = peaks[0]
t_peaks = t[i_peaks]
t_diff = t_peaks[1:] - t_peaks[:-1]
t_diff = t_diff[10:]
T = np.mean(t_diff)
J_y = (m * g * r/((2 * np.pi / T) ** 2)) - m * r ** 2 # <-- FIXME
print(J_y)

7.457926030303888e-05


### 1.4 Perform z-axis moment of inertia calculation

In [7]:
data = load_hardware_data(
    'z_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)
t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])
peaks = find_peaks(w_z, prominence=0.025)
i_peaks = peaks[0]
t_peaks = t[i_peaks]
t_diff = t_peaks[1:] - t_peaks[:-1]
T = np.mean(t_diff)
J_z = (m * g * r_z/((2 * np.pi / T) ** 2)) - m * r_z ** 2 # <-- FIXME
print(J_z)

4.5815682178722596e-05


# 2. Planar Problem

The dynamical model for the planar system is now derived.

## 2.1 Variable Setup

The new information added here are the states of the pendulum, $\alpha$ and $\dot{\alpha}$ which represent the angular position and angular velocity of the pendulum.

Define states

In [8]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# states of the pendulum
alpha, alpha_dot = sym.symbols('alpha, alpha_dot')

Define inputs

In [9]:
# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

Define parameters.
$m_{pen}$ is the mass of the pendulum and $l_{pen}$ is the distance of the center of mass of the pendulum from its base.

In [10]:
m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')

Create linear and angular velocity vectors (in coordinates of the body frame).

In [11]:
v_01in1 = sym.Matrix([v_x, v_y, v_z])
w_01in1 = sym.Matrix([w_x, w_y, w_z])

Create moment of inertia matrix (in coordinates of the body frame).

In [12]:
J_in1 = sym.diag(J_x, J_y, J_z)

### 2.2 Deriving the Kinematics of the Pendulum

Generalized Co-ordinates: $x$, $z$ (in the world frame), $\theta$ (pitch) and $\alpha$ (angle of the drone from the world $z_0$)

This derivation deals with the planar problem that ignores the 3D effects of the roll, yaw and y-direction movement of the pendulum. The equations of motion from this are then added to the existing equations of motion.

![pendulum_kin](Inverted_Pendulum_Kinematics.jpg)

Dynamic copies of the symbolic variables are set up below (symbolic variables that are functions of time). The Lagrangian of this system are then calculated, and then Lagrangian equations are used to derive the equations of motion of the joint system.

In [13]:
dyn_o_x, dyn_o_z, dyn_alpha, dyn_theta = dynamicsymbols(r'o_x o_z \alpha \theta')
dyn_o_xd, dyn_o_zd, dyn_alphad, dyn_thetad = dynamicsymbols(r'o_x o_z \alpha \theta', 1)
dyn_o_xdd, dyn_o_zdd, dyn_alphadd, dyn_thetadd = dynamicsymbols(r'o_x o_z \alpha \theta', 2)
L = 1/2*m*(dyn_o_xd**2 + dyn_o_zd**2) + 1/2*J_y*(dyn_thetad**2) + 1/2*m_pen*(dyn_o_xd**2 + dyn_o_zd**2 + l_pen**2 * dyn_alphad**2 + 2*l_pen*dyn_alphad*(dyn_o_xd*sym.cos(dyn_alpha) - dyn_o_zd*sym.sin(dyn_alpha))) - m*g*dyn_o_z - m_pen*g*(dyn_o_z + l_pen*sym.cos(dyn_alpha))
dyn_f_z, dyn_tau_y = sym.symbols(r'f_z, \tau_y')
t = sym.Symbol('t')

Equations = sym.Matrix([[sym.simplify(sym.diff(sym.diff(L, dyn_o_xd), t) - sym.diff(L, dyn_o_x) - dyn_f_z*sym.sin(dyn_theta))],
             [sym.simplify(sym.diff(sym.diff(L, dyn_o_zd), t) - sym.diff(L, dyn_o_z) - dyn_f_z*sym.cos(dyn_theta))],
             [sym.simplify(sym.diff(sym.diff(L, dyn_alphad), t) - sym.diff(L, dyn_alpha))],
             [sym.simplify(sym.diff(sym.diff(L, dyn_thetad), t) - sym.diff(L, dyn_theta) - dyn_tau_y)]])

Equations

Matrix([
[                   -f_z*sin(\theta(t)) + 1.0*m*Derivative(o_x(t), (t, 2)) + m_pen*(-l_pen*sin(\alpha(t))*Derivative(\alpha(t), t)**2 + l_pen*cos(\alpha(t))*Derivative(\alpha(t), (t, 2)) + Derivative(o_x(t), (t, 2)))],
[-f_z*cos(\theta(t)) + g*m + g*m_pen + 1.0*m*Derivative(o_z(t), (t, 2)) - 1.0*m_pen*(l_pen*sin(\alpha(t))*Derivative(\alpha(t), (t, 2)) + l_pen*cos(\alpha(t))*Derivative(\alpha(t), t)**2 - Derivative(o_z(t), (t, 2)))],
[                                                          l_pen*m_pen*(-g*sin(\alpha(t)) + l_pen*Derivative(\alpha(t), (t, 2)) - sin(\alpha(t))*Derivative(o_z(t), (t, 2)) + cos(\alpha(t))*Derivative(o_x(t), (t, 2)))],
[                                                                                                                                                                         1.0*J_y*Derivative(\theta(t), (t, 2)) - \tau_y]])

The equations of motion above are solved to find the expressions for $\ddot{o}_x$, $\ddot{o}_z$, $\ddot{\alpha}$, and $\ddot{\theta}$. The dynamic symbols are then substituted for the state and input variables set before.

In [14]:
# Solving for the second derivatives
ox_ddot_with_alphaddot = sym.solve(Equations[0,0], dyn_o_xdd)[0]
oz_ddot_with_alphaddot = sym.solve(Equations[1,0], dyn_o_zdd)[0]
alpha_ddot_eqn = Equations[2].subs([(dyn_o_xdd, ox_ddot_with_alphaddot), (dyn_o_zdd, oz_ddot_with_alphaddot)])
alpha_ddot_soln = sym.simplify(sym.solve(alpha_ddot_eqn, dyn_alphadd)[0])
theta_ddot_soln = sym.solve(Equations[3,0], dyn_thetadd)[0]
new_ox_ddot = sym.simplify(ox_ddot_with_alphaddot.subs(dyn_alphadd, alpha_ddot_soln))
new_oz_ddot = sym.simplify(oz_ddot_with_alphaddot.subs(dyn_alphadd, alpha_ddot_soln))

# Substituting the standard symbolic variables
theta_ddot_soln = theta_ddot_soln.subs(dyn_tau_y, tau_y)
new_ox_ddot = new_ox_ddot.subs([(dyn_f_z, f_z), (dyn_theta, theta), (dyn_alphad, alpha_dot)])
new_ox_ddot = new_ox_ddot.subs(dyn_alpha, alpha)
new_oz_ddot = new_oz_ddot.subs([(dyn_f_z, f_z), (dyn_theta, theta), (dyn_alphad, alpha_dot)])
new_oz_ddot = new_oz_ddot.subs(dyn_alpha, alpha)
alpha_ddot_soln = alpha_ddot_soln.subs([(dyn_alpha, alpha), (dyn_theta, theta), (dyn_f_z, f_z)])

The planar problem is set up to be linearized with the 4 variables mentioned above and their time derivatives.

In [15]:
o_x_dot, o_z_dot, theta_dot = sym.symbols(r'o_x_dot, o_z_dot, theta_dot')

f_sym_planar = sym.Matrix.vstack(sym.Matrix([o_x_dot]),
                          sym.Matrix([new_ox_ddot]),
                          sym.Matrix([o_z_dot]),
                          sym.Matrix([new_oz_ddot]),
                          sym.Matrix([theta_dot]),
                          sym.Matrix([theta_ddot_soln]),
                          sym.Matrix([alpha_dot]),
                          sym.Matrix([alpha_ddot_soln])
                          )
f_sym_planar

Matrix([
[                                                                                                                             o_x_dot],
[               (-f_z*m_pen*sin(alpha - theta)*cos(alpha) + m*(alpha_dot**2*l_pen*m_pen*sin(alpha) + f_z*sin(theta)))/(m*(m + m_pen))],
[                                                                                                                             o_z_dot],
[(f_z*m_pen*sin(alpha)*sin(alpha - theta) + m*(alpha_dot**2*l_pen*m_pen*cos(alpha) + f_z*cos(theta) - g*m - g*m_pen))/(m*(m + m_pen))],
[                                                                                                                           theta_dot],
[                                                                                                                           tau_y/J_y],
[                                                                                                                           alpha_dot],
[                                      

In [16]:
s_planar = [o_x, o_x_dot, o_z, o_z_dot, theta, theta_dot, alpha, alpha_dot]
i_planar = [tau_y, f_z]
p_planar = [m, J_y, g, m_pen, l_pen]
s_with_des_planar = [o_x, o_z]

In [17]:
f_planar = sym.lambdify(s_planar + i_planar + p_planar, f_sym_planar)
f_planar

<function _lambdifygenerated(o_x, o_x_dot, o_z, o_z_dot, theta, theta_dot, alpha, alpha_dot, tau_y, f_z, m, J_y, g, m_pen, l_pen)>

The values for the constants are given but only for the planar case so the full system derivation that follows can still use the symbolic variables.

In [18]:
# Mass
m_planar = 0.044

# Principle moments of inertia
J_y_planar = 1.60e-05

# Acceleration of gravity
g_planar = 9.81

# Pendulum parameters
l_pen_planar = 150e-3 # maybe m
m_pen_planar = 0.006

Checking if the equillibrium point is valid

In [19]:
p_planar_eq = [m_planar, J_y_planar, g_planar, m_pen_planar, l_pen_planar]
s_planar_eq = [0., 0., 0., 0., 0., 0., 0., 0.]
i_planar_eq = [0., g_planar*(m_planar + m_pen_planar)]

In [20]:
print(f_planar(*s_planar_eq, *i_planar_eq, *p_planar_eq))

[[ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [-6.9388939e-16]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]]


The system is then linearized into the state space model at this equilibrium point.

In [21]:
A_planar_sym = f_sym_planar.jacobian(s_planar)
B_planar_sym = f_sym_planar.jacobian(i_planar)

A_planar_num = sym.lambdify(s_planar + i_planar + p_planar, A_planar_sym)
B_planar_num = sym.lambdify(s_planar + i_planar + p_planar, B_planar_sym)

A_planar = A_planar_num(*s_planar_eq, *i_planar_eq, *p_planar_eq)
B_planar = B_planar_num(*s_planar_eq, *i_planar_eq, *p_planar_eq)

In [22]:
A_planar_str = np.array2string(A_planar,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_planar_str}')

A = [[ 0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00 11.15  0.00 -1.34  0.00]
     [ 0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00 -0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00]
     [ 0.00  0.00  0.00  0.00 -74.32  0.00 74.32  0.00]]


In [23]:
B_planar_str = np.array2string(B_planar,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)
print(f'B = {B_planar_str}')

B = [[      0.00       0.00]
     [      0.00       0.00]
     [      0.00       0.00]
     [      0.00      20.00]
     [      0.00       0.00]
     [  62500.00       0.00]
     [      0.00       0.00]
     [      0.00       0.00]]


We can check if this planar problem is controllable or not by looking at the controllability matrix for the above state-space model.

In [24]:
W_planar = np.block([[B_planar, A_planar @ B_planar, A_planar @ A_planar @ B_planar, np.linalg.matrix_power(A_planar, 3) @ B_planar, np.linalg.matrix_power(A_planar, 4) @ B_planar, np.linalg.matrix_power(A_planar, 5) @ B_planar, np.linalg.matrix_power(A_planar, 6) @ B_planar, np.linalg.matrix_power(A_planar, 7) @ B_planar]])

np.linalg.matrix_rank(W_planar) # If 8, the system is controllable

8

This shows that the planar problem and the system above is controllable. In the next section, these equations will be used in the full system state-space model to derive a controller.

# 3. Derive full-system state-space model

## 3.1 Define kinematics of orientation

### 3.1.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [25]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [26]:
R_1in0 = Rz * Ry * Rx

In [27]:
R_1in0

Matrix([
[cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)],
[sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)],
[        -sin(theta),                              sin(phi)*cos(theta),                               cos(phi)*cos(theta)]])

### 3.1.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [28]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [59]:
N = sym.simplify(Ninv.inv())
Ninv

Matrix([
[        -sin(theta),         0, 1],
[sin(phi)*cos(theta),  cos(phi), 0],
[cos(phi)*cos(theta), -sin(phi), 0]])

## 3.2 Define equations of motion

We once again consider the equations of motion of the full-system, and not just the planar problem.

Forces.

In [30]:
f_in1 = R_1in0.T * sym.Matrix([0, 0, -m * g]) + sym.Matrix([0, 0, f_z])

Torques.

In [31]:
tau_in1 = sym.Matrix([tau_x, tau_y, tau_z])

In [32]:
f_in1

Matrix([
[               g*m*sin(theta)],
[     -g*m*sin(phi)*cos(theta)],
[f_z - g*m*cos(phi)*cos(theta)]])

In [33]:
tau_in1

Matrix([
[tau_x],
[tau_y],
[tau_z]])

The block below calculates the new system of forces in the body frame that are taken from the expressions for $\ddot{o}_x$ and $\ddot{o}_z$. There is potential for error here, and could be resulting in the lack of full controllability of the complete system. So, this wasn't used in the final equations of motion.

In [34]:
# new_f_in1 = R_1in0.T * sym.Matrix([(m)*new_ox_ddot, 0, (m)*new_oz_ddot])
# sym.simplify(new_f_in1)

Create equations of motion. Here, we include the dynamics of the pendulum in the new terms for $\alpha$ and $\dot{\alpha}$

In [35]:
f_sym = sym.Matrix.vstack(R_1in0 * v_01in1,
                          N * w_01in1,
                          (1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)),
                          J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1)),
                          sym.Matrix([alpha_dot]),
                          sym.Matrix([alpha_ddot_soln])
                          )

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

for states

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \\ \alpha \\ \dot{\alpha} \end{bmatrix},
$$

inputs
$$
i = \begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix},
$$

and parameters
$$
p = \begin{bmatrix} m \\ J_x \\ J_y \\ J_z \\ g \\ m_{pen} \\ l_{pen} \end{bmatrix}.
$$

In [36]:
f_sym

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                (g*m*sin(theta) + m*v_y*w_z - m*v_z*w_y)/m

## 3.3 Choose equilibrium point

An equilibrium point of the nonlinear system is a choice of states $s_\text{eq}$ and inputs $i_\text{eq}$ - along with constant parameters $p_\text{eq}$ - for which

$$0 = f(s_\text{eq}, i_\text{eq}, p_\text{eq}).$$

Create a list of states, inputs, and parameters as symbolic variables.

In [37]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, alpha, alpha_dot]
i = [tau_x, tau_y, tau_z, f_z]
p = [m, J_x, J_y, J_z, g, m_pen, l_pen]

Create a list of states to track as symbolic variables. These are states whose desired values will be specified by a client.

In [38]:
s_with_des = [o_x, o_y, o_z]

Create a function that evaluates $f(\cdot)$ at particular values of $s$, $i$, and $p$.

In [39]:
f = sym.lambdify(s + i + p, f_sym)

In [40]:
f

<function _lambdifygenerated(o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, alpha, alpha_dot, tau_x, tau_y, tau_z, f_z, m, J_x, J_y, J_z, g, m_pen, l_pen)>

Define constants.

In [41]:
# Mass
m = 0.044

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
g = 9.81

# Pendulum parameters
l_pen = 150e-3 # maybe m
m_pen = 0.006

Create a list of parameter values in the **same order** as the symbolic list. These are the parameter estimates we found in our experiments. They are not choices. (We use the subscript `_eq` to be consistent with what follows, and could say "parameter values *at equilibrium*," but don't be misled. These parameter values are *given* and are *constant* - again, they aren't choices.)

In [42]:
p_eq = [m, J_x, J_y, J_z, g, m_pen, l_pen]

Create a list of state and input values at equilibrium in the **same order** as the symbolic lists.

In [43]:
s_eq = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
i_eq = [0., 0., 0., g*(m)] # Notice that the mass of the pendulum is now factored into the equilibrium point

Evaluate the equations of motion at the equilibrium point - if it actually *is* an equilibrium point, then the result should be an array of zeros:

In [44]:
print(f(*s_eq, *i_eq, *p_eq))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


Note that this equilibrium point would remain an equilibrium point for any choice of `o_x`, `o_y`, and `o_z` - that is one thing that allows the controller to track desired values of these variables.

## 3.4 Compute A and B

We want to find

$$
A = \frac{\partial f}{\partial s}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}
\qquad\text{and}\qquad
B = \frac{\partial f}{\partial i}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}.
$$

First, we compute each Jacobian (i.e., each matrix of partial derivatives) in symbolic form.

In [45]:
A_sym = f_sym.jacobian(s)
B_sym = f_sym.jacobian(i)

Then, we create functions that allow us to evaluate these Jacobians at particular values of $s$, $i$, and $p$.

In [46]:
A_num = sym.lambdify(s + i + p, A_sym)
B_num = sym.lambdify(s + i + p, B_sym)

Finally, we plug in our equilibrium point.

In [47]:
A = A_num(*s_eq, *i_eq, *p_eq)
B = B_num(*s_eq, *i_eq, *p_eq)

Show $A$ (formatted nicely).

In [48]:
A_str = np.array2string(A,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_str}')

A = [[ 0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00 -0.00  0.00 -0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  1.00 -0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  9.81  0.00  0.00  0.00 -0.00  0.00 -0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00 -9.81 -0.00  0.00  0.00  0.00  0.00 -0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00 -0.00  0.00 -0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]

Show $B$ (formatted nicely).

In [49]:
B_str = np.array2string(B,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)
B
print(f'B = {B_str}')

B = [[      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00      22.73]
     [  63694.27       0.00       0.00       0.00]
     [      0.00   62500.00       0.00       0.00]
     [      0.00       0.00   48780.49       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]]


The state-space system is described by

$$ \dot{x} = Ax + Bu $$

where

$$ x = s - s_\text{eq} $$

and

$$ u = i - i_\text{eq}. $$

$A$ and $B$ would remain the same for any choice of `o_x`, `o_y`, and `o_z` - that is the other thing that allows the controller to track desired values of these variables.

Check controllability:

In [50]:
W = np.block([[B, A @ B, A @ A @ B, np.linalg.matrix_power(A, 3) @ B, np.linalg.matrix_power(A, 4) @ B, np.linalg.matrix_power(A, 5) @ B, np.linalg.matrix_power(A, 6) @ B, np.linalg.matrix_power(A, 7) @ B, np.linalg.matrix_power(A, 8) @ B, np.linalg.matrix_power(A, 9) @ B, np.linalg.matrix_power(A,10) @ B, np.linalg.matrix_power(A, 11) @ B, np.linalg.matrix_power(A, 12) @ B, np.linalg.matrix_power(A, 13) @ B]])

np.linalg.matrix_rank(W) # If 14, the system is controllable


12

From this it seems like the system is not fully controllable. The reason for this is probably the inaccuracy in new forces term derived from the planar problem and put into the full system. This is a point of further investigation.

# 4. Design method of power distribution


Define constants.

In [51]:
k_F = 2.04e-06
k_M = 6.33e-09
l = .0325 

Defines the matrix $P$ that maps motor power commands ($m_1$, $m_2$, $m_3$, $m_4$) to inputs ($\tau_x$, $\tau_y$, $\tau_z$, $f_z$).

In [52]:
P = np.array([[ -l * k_F, -l * k_F,  l * k_F,  l * k_F  ],
              [ -l * k_F, l * k_F,   l * k_F,  -l * k_F ],
              [ -k_M,     k_M,       -k_M,     k_M      ],
              [ k_F,      k_F,       k_F,      k_F      ]])

Computes the matrix $P^{-1}$ that maps inputs to motor power commands.

In [53]:
Pinv = linalg.inv(P)

Print code that implements the method of power distribution in python (for simulation).

In [54]:
def export_power_distribution(Pinv,
                              limiter='self.limitUint16',
                              decimals=1,
                              suffix='',
                              line_ending=''):
    """
    Pinv is a 4 x 4 matrix that maps inputs (tau_x, tau_y, tau_z, f_z)
        to motor power commands (m_1, m_2, m_3, m_4)
    limiter is the name of the function to apply that ensures each
        motor power command is valid (i.e., an integer within bounds),
        for example "limitUint16" when exporting to C
    decimals is the number of decimals to include when printing
        each value
    suffix is the character (if any) to print after each number,
        for example 'f' to indicate a "float" when exporting to C
    line_ending is the character (if any) to print after each
        line, for example ';' when exporting to C
    """
    
    i_name = ['tau_x', 'tau_y', 'tau_z', 'f_z']
    m_name = ['m_1', 'm_2', 'm_3', 'm_4']
    for row in range(len(m_name)):
        input_string = ''
        for col in range(len(i_name)):
            k = Pinv[row, col]
            if not np.isclose(k, 0.):
                if (k > 0) and input_string:
                    input_string += ' +'
                n = i_name[col]
                input_string += f' {k:.{decimals}f}{suffix} * {n}'
        print(f'{m_name[row]} = {limiter}({input_string} ){line_ending}')

export_power_distribution(Pinv)

m_1 = self.limitUint16( -3770739.1 * tau_x -3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
m_2 = self.limitUint16( -3770739.1 * tau_x + 3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
m_3 = self.limitUint16( 3770739.1 * tau_x + 3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
m_4 = self.limitUint16( 3770739.1 * tau_x -3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )


Print code that implements the method of power distribution in C (for hardware).

In [55]:
export_power_distribution(
    Pinv,
    limiter='limitUint16',
    suffix='f',
    line_ending=';',
)

m_1 = limitUint16( -3770739.1f * tau_x -3770739.1f * tau_y -39494470.8f * tau_z + 122549.0f * f_z );
m_2 = limitUint16( -3770739.1f * tau_x + 3770739.1f * tau_y + 39494470.8f * tau_z + 122549.0f * f_z );
m_3 = limitUint16( 3770739.1f * tau_x + 3770739.1f * tau_y -39494470.8f * tau_z + 122549.0f * f_z );
m_4 = limitUint16( 3770739.1f * tau_x -3770739.1f * tau_y + 39494470.8f * tau_z + 122549.0f * f_z );


# 5. Control design

This controller uses a linear state feedback of the form

$$u = - K (x - x_\text{des})$$

where the gain matrix $K$ is chosen by solving an infinite-horizon LQR problem (e.g., with the helper function `lqr`) and where

$$x_\text{des} = \begin{bmatrix} o_{x, \text{des}} \\ o_{y, \text{des}} \\ o_{z, \text{des}} \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0\end{bmatrix}.$$

The LQR problem is defined by the choice of weights $Q$ and $R$. Both must be square matrices, $Q$ must be positive semidefinite, and $R$ must be positive definite. Both $Q$ and $R$ are almost always chosen to be diagonal. $Q$ penalizes non-zero states (i.e., non-zero values of $x$) and that $R$ penalizes non-zero inputs (i.e., non-zero values of $u$).

In [56]:
Q = np.diag([
    0.01, # x
    7., # y
    5., # z
    1., # psi (yaw)
    20., # theta (pitch)
    1., # phi (roll)
    0.02,
    0.8,
    1.,
    1.,
    5.,
    1.,
    50., #alpha
    2. #alpha_dot
])

R = np.diag([
    1E6, #tau_x
    1E9,#tau_y
    1E6,
    10,
])


Find $K$ for the chosen $Q$ and $R$.

Here is a function that solves the linear quadratic regulator (LQR) problem - i.e., that finds the matrix $K$ for which

$$u(t) = - K x(t)$$

is the optimal solution to

$$
\begin{align*}
\underset{u_{[t_{0}, \infty)}}{\text{minimize}} &\qquad \int_{t_{0}}^{\infty}\left( x(t)^{T}Qx(t)+u(t)^{T}Ru(t)\right)dt \\
\text{subject to} &\qquad \dot{x}(t) = Ax(t)+Bu(t), \quad x(t_{0})=x_{0}.
\end{align*}
$$

In [57]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

In [58]:
K = lqr(A, B, Q, R)

K

KeyboardInterrupt: 

Show $K$ (formatted nicely).

In [ ]:
K_str = np.array2string(K,
                        formatter={'float_kind': lambda x: f'{x:6.3f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'K = {K_str}')

K = [[-0.000 -0.003  0.000  0.000  0.000  0.007 -0.000 -0.002  0.000  0.001  0.000  0.000 -0.000 -0.000]
     [-0.000 -0.000 -0.000  0.000  0.004  0.000 -0.000 -0.000  0.000  0.000  0.000 -0.000 -0.005 -0.001]
     [ 0.000 -0.000  0.000  0.001 -0.000  0.000  0.000 -0.000  0.000  0.000 -0.000  0.001  0.000  0.000]
     [ 0.000 -0.000  0.707  0.000  0.000  0.000  0.000 -0.000  0.403  0.000  0.000  0.000 -0.000 -0.000]]


Print code that implements the controller.

In [ ]:
def export_controller(K, s, i, s_with_des, i_eq,
                      decimals=8,
                      suffix='',
                      line_ending=''):
    """
    K is a gain matrix, of size m x n
    s is a list of states as symbolic variables, of length n
    i is a list of inputs as symbolic variables, of length m
    s_with_des is a list of states that have desired values, as
        symbolic variables - if there are no such states, then
        this should be an empty list []
    i_eq is a list of equilibrium values of inputs, of length m
    decimals is the number of decimals to include when printing
        each value
    suffix is the character (if any) to print after each number,
        for example 'f' to indicate a "float" when exporting to C
    line_ending is the character (if any) to print after each
        line, for example ';' when exporting to C
    """
    
    s_name = [scur.name for scur in s]
    i_name = [icur.name for icur in i]
    for row in range(len(i_name)):
        input_string = ''
        for col in range(len(s_name)):
            k = K[row, col]
            if not np.isclose(k, 0.):
                if (k < 0) and input_string:
                    input_string += ' +'
                if s[col] in s_with_des:
                    n = f'({s_name[col]} - {s_name[col]}_des)'
                else:
                    n = s_name[col]
                input_string += f' {-k:.{decimals}f}{suffix} * {n}'
        if not np.isclose(i_eq[row], 0.):
            if (i_eq[row] > 0) and input_string:
                input_string += ' +'
            input_string += f' {i_eq[row]:.{decimals}f}{suffix}'
        print(f'{i_name[row]} ={input_string}{line_ending}')


# For python simulation
export_controller(
    K,               # the gain matrix
    s,               # list of states as symbolic variables
    i,               # list of inputs as symbolic variables
    s_with_des,      # list of states that have desired values as symbolic variables
    i_eq,            # list of equilibrium values of inputs
)

# For C
# export_controller(
#     K,               # the gain matrix
#     s,               # list of states as symbolic variables
#     i,               # list of inputs as symbolic variables
#     s_with_des,      # list of states that have desired values as symbolic variables
#     i_eq,            # list of equilibrium values of inputs
#     suffix='f',      # character to print after each number (indicates a "float")
#     line_ending=';'  # character to print after each line
# )


tau_x = 0.00264575 * (o_y - o_y_des) -0.00676379 * phi + 0.00210912 * v_y -0.00110108 * w_x
tau_y = 0.00000316 * (o_x - o_x_des) -0.00442957 * theta + 0.00001596 * v_x -0.00038307 * w_y + 0.00479345 * alpha + 0.00059377 * alpha_dot
tau_z = -0.00100000 * psi -0.00102029 * w_z
f_z = -0.70710678 * (o_z - o_z_des) -0.40277214 * v_z + 0.43164000


# 6. Simulation

The following section runs a simulation to test the controller. This simulator is based on the AE 483 Autonomous Systems Lab controller. The drone model is modified to include an inverted pendulum, and the script is modified to allow the measurement of a pendulum angle.

In [ ]:
from IPython.display import display, IFrame, HTML
import ae483_drone

## 6.1 Run Simulation

### 6.1.1 Create an instance of the simulation

In [ ]:
simulator = ae483_drone.Simulator(
    display=True,
    width=640,
    height=480,
)

### 6.1.2 Client enables smooth motion

In [ ]:
class RobotClient:
    def __init__(self):
        pass
    
    def run(self, t):
        o_x = 0.
        o_y = 0.
        o_z = 0.3
        psi = 0.
        
        return {
            'o_x': o_x,
            'o_y': o_y,
            'o_z': o_z,
            'psi': psi,
        }

### 6.1.3 Create controller class (with force, torques, and motor commands)

In [ ]:
class RobotController:
    def __init__(self):
        pass

    def limitUint16(self, m):
        """
        This function returns the closest integer to "m" in the
        range [0, 65535]. It is called "limitUint16" because an
        "unsigned 16-bit integer" is limited to this range.
        """
        m = np.round(m, decimals=0)
        if m < 0:
            m = 0
        elif m > 65535:
            m = 65535
        return m

    def run(self, state, setpoint):
        # Parse state
        o_x = state['o_x']
        o_y = state['o_y']
        o_z = state['o_z']
        psi = state['psi']
        theta = state['theta']
        phi = state['phi']
        v_x = state['v_x']
        v_y = state['v_y']
        v_z = state['v_z']
        w_x = state['w_x']
        w_y = state['w_y']
        w_z = state['w_z']
        alpha = state['alpha']
        alpha_dot = state['alpha_dot']
        # Parse setpoint
        o_x_des = setpoint['o_x']
        o_y_des = setpoint['o_y']
        o_z_des = setpoint['o_z']
        
        tau_x = 0.00264575 * (o_y - o_y_des) -0.00666549 * phi + 0.00209651 * v_y -0.00109968 * w_x
        tau_y = 0.00000316 * (o_x - o_x_des) -0.00403257 * theta + 0.00001570 * v_x -0.00035924 * w_y + 0.00438403 * (alpha + theta) + 0.00054324 * alpha_dot
        tau_z = -0.00100000 * psi -0.00102029 * w_z
        f_z = -0.70710678 * (o_z - o_z_des) -0.40277214 * v_z + 0.43164000
        
        # tau_x = 0.00264575 * (o_y - o_y_des) -0.00676379 * phi + 0.00210912 * v_y -0.00110108 * w_x
        # tau_y = 0.00000316 * (o_x - o_x_des) -0.00442957 * theta + 0.00001596 * v_x -0.00038307 * w_y + 0.00479345 * alpha + 0.00059377 * alpha_dot
        # tau_z = -0.00100000 * psi -0.00102029 * w_z
        # f_z = -0.70710678 * (o_z - o_z_des) -0.40277214 * v_z + 0.43164000
        
        m_1 = self.limitUint16( -3770739.1 * tau_x -3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_2 = self.limitUint16( -3770739.1 * tau_x + 3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        m_3 = self.limitUint16( 3770739.1 * tau_x + 3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_4 = self.limitUint16( 3770739.1 * tau_x -3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        
        return m_1, m_2, m_3, m_4

### 6.1.4 Reset simulation by clearing drones

In [ ]:
simulator.clear_drones()

### 6.1.5 Add drone to simulation

In [ ]:
simulator.add_drone(
    'my_drone',
    RobotClient,
    RobotController,
    rgba=[1., 0., 1., 1.],
    m=0.039,  
    J_x=1e-5, 
    J_y=1e-5, 
    J_z=2e-5, 
    g=9.81,
    l=0.035,   
    k_F=2e-6,  
    k_M=6.33e-09, 
)

### 6.1.6 Set initial drone state

In [ ]:
simulator.set_state(
    'my_drone',
    {
        'o_x': 0.,
        'o_y': 0.,
        'o_z': 0.3,
        'psi': 1.57,
        'theta': 0.05,
        'phi': 0.,
        'v_x': 0.,
        'v_y': 0.,
        'v_z': 0.,
        'w_x': 0.,
        'w_y': 0.,
        'w_z': 0.,
        'alpha': -0.2
    },
)

### 6.1.7 Set simulation camera view

In [ ]:
# simulator.set_camera_target('my_drone')
# simulator.set_camera_yaw(90.0)
# simulator.camera_sideview()
simulator.camera_topview()

### 6.1.8 Run simulation

In [ ]:
simulator.run(
    max_time=20.0,
    data_filename='pendulum_simulation_test.json',
    video_filename=None,
)

Completed 2000 time steps in 35.5891 seconds (56.1970 time steps per second)


## 6.2 Plot results from simulation

### 6.2.1 Define function to load data from simulation

In [ ]:
def load_simulation_data(filename, dronename):
    # load data for all drones
    with open(filename, 'r') as f:
        data = json.load(f)
    
    # get data for one drone only
    data = data[dronename]
    
    # convert lists to numpy arrays
    for key in data.keys():
        if isinstance(data[key], dict):
            for subkey in data[key].keys():
                data[key][subkey] = np.array(data[key][subkey])
        else:
            data[key] = np.array(data[key])
    
    # return the data
    return data

### 6.2.2 Load and process simulation JSON data

In [ ]:
data = load_simulation_data(
    'pendulum_simulation_test.json', 
    'my_drone',              
)

# time
t = data['t']

# position
o_x = data['state']['o_x']
o_y = data['state']['o_y']
o_z = data['state']['o_z']

# desired position
o_x_des = data['setpoint']['o_x']
o_y_des = data['setpoint']['o_y']
o_z_des = data['setpoint']['o_z']

# orientation
psi = data['state']['psi']
theta = data['state']['theta']
phi = data['state']['phi']

# pendulum data
alpha = data['state']['alpha']
alpha_dot = data['state']['alpha_dot']

# motor power commands
m_1 = data['power']['m_1']
m_2 = data['power']['m_2']
m_3 = data['power']['m_3']
m_4 = data['power']['m_4']

# Compute error in position (this is a function of time).

o_x_err = o_x - o_x_des
o_y_err = o_y - o_y_des
o_z_err = o_z - o_z_des

### 6.2.3 Plot simulation results from loaded data

In [ ]:
# Plot position error, orientation, pendulum pose and motor power commands

fig, (ax_pos, ax_ori, ax_pow, ax_pen, ax_dot_pen) = plt.subplots(5, 1, figsize=(12, 12), sharex=True)
ax_pos.plot(t, o_x_err, label='x position error')
ax_pos.plot(t, o_y_err, label='y position error')
ax_pos.plot(t, o_z_err, label='z position error')
ax_pos.legend()
ax_pos.grid()
ax_ori.plot(t, psi, label='psi')
ax_ori.plot(t, theta, label='theta')
ax_ori.plot(t, phi, label='phi')
ax_ori.legend()
ax_ori.grid()
ax_pow.plot(t, m_1, label='m_1')
ax_pow.plot(t, m_2, label='m_2')
ax_pow.plot(t, m_3, label='m_3')
ax_pow.plot(t, m_4, label='m_4')
ax_pow.legend()
ax_pow.grid()
ax_pen.plot(t, alpha, label='alpha')
ax_pen.legend()
ax_pen.grid()
ax_dot_pen.plot(t, alpha_dot, label='alpha_dot')
ax_dot_pen.legend()
ax_dot_pen.grid()
ax_pow.set_xlabel('time (s)')

# 7. Real-World Analysis

### 7.1 Define function to load data from hardware

In [ ]:
def load_hardware_data(filename, t_min_offset=0, t_max_offset=0):
    # load raw data
    with open(filename, 'r') as f:
        data = json.load(f)

    # convert lists to numpy arrays
    for val in data.values():
        for key in val.keys():
            val[key] = np.array(val[key])

    # create an array of times at which to subsample
    t_min = -np.inf
    t_max = np.inf
    for key, val in data.items():
        t_min = max(t_min, val['time'][0])
        t_max = min(t_max, val['time'][-1])
    t_min += t_min_offset * 1000
    t_max -= t_max_offset * 1000
    nt = int(1 + np.floor((t_max - t_min) / 10.))
    t = np.arange(0, 10 * nt, 10) / 1000.
    resampled_data = {'time': t}

    # resample raw data with linear interpolation
    for k, v in data.items():
        f = interp1d((v['time'] - t_min) / 1000., v['data'])
        resampled_data[k] = f(t)
        
    # return the resampled data
    return resampled_data

### 7.2 Load and plot hardware data

This section reproduces the real-world test plots from the report in sequence.

In [ ]:
json_list = ['FlightData/Flight_1.json', 'FlightData/Flight_2.json', 'FlightData/Flight_4.json']

for json_path in json_list:
    data = load_hardware_data(
        json_path, # <-- replace with name of file with hardware data
        t_min_offset=3.5,      # <-- (optional) replace with how many seconds of data to ignore at start
        t_max_offset=20.75,      # <-- (optional) replace with how many seconds of data to ignore at end
    )

    t = data['time']
    alpha = data['ae483log.alpha']
    ox = data['ae483log.o_x']
    oz = data['ae483log.o_z']

    fig, (angle, x_pos, z_pos) = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
    angle.plot(t, np.degrees(alpha))
    angle.grid()
    angle.set_ylabel('Alpha (deg)')
    x_pos.plot(t, ox)
    x_pos.grid()
    x_pos.set_ylabel('X Position (m)')
    z_pos.plot(t, oz)
    z_pos.grid()
    z_pos.set_xlabel("Time (s)")
    z_pos.set_ylabel("Z Position (m)")

### 7.3 Define function to show Media Space videos

In [ ]:
def mediaspace(url, width=640, height=397):
    video_id = url.rsplit('/', 1)[-1]
    src = f'https://mediaspace.illinois.edu/embed/secure/iframe/entryId/{video_id}?st=0'
    display(IFrame(src, width=width, height=height))

### 7.4 Show videos

The videos below showcase some of the milestone tests that showcase important behaviors of the controller's the team tested. Videos 1 and 4 just use the direct feedback of $\alpha$ into $\tau_y$, whereas videos 2 and 3 use the LQR controller derived in this notebook. 

In [ ]:
video_list = ['https://mediaspace.illinois.edu/media/t/1_gjr2ykpm', 'https://mediaspace.illinois.edu/media/t/1_86zin1mc',
              'https://mediaspace.illinois.edu/media/t/1_qrr8klzf', 'https://mediaspace.illinois.edu/media/t/1_tvrlf9de']
for video in video_list:
    mediaspace(video)
